# Package Example

## Imports

In [1]:
from plants_sm.data_structures.dataset.multi_input_dataset import MultiInputDataset
from plants_sm.data_standardization.proteins.standardization import ProteinStandardizer
from plants_sm.data_standardization.compounds.deepmol_standardizers import DeepMolStandardizer
from plants_sm.featurization.proteins.bio_embeddings.word2vec import Word2Vec
from plants_sm.featurization.compounds.deepmol_descriptors import DeepMolDescriptors
from torch import nn
from sklearn.metrics import f1_score
from torch.optim import Adam
from plants_sm.models.constants import BINARY
from plants_sm.models.pytorch_model import PyTorchModel
from plants_sm.models.interaction.baseline_model import BaselineModel
from plants_sm.tokenisation.compounds.smilespe import AtomLevelTokenizer
import pandas as pd


from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [2]:
multi_input_dataset_csv = "./data/aminotransferase_binary.csv"
pd.read_csv(multi_input_dataset_csv).head()

,ids,SEQ,SUBSTRATES,LogSpActivity
0,0,MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...,C(C(=O)O)N,0
1,1,MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...,C[C@@H](C(=O)O)N,1
2,2,MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...,CC(C)[C@@H](C(=O)O)N,0
3,3,MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...,CC(C)C[C@@H](C(=O)O)N,1
4,4,MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVL...,CC[C@H](C)[C@@H](C(=O)O)N,0


## Read datasets

In [3]:
multi_input_dataset_csv = "./multi_input_dataset.csv"

multi_input_dataset = MultiInputDataset.from_csv(file_path=multi_input_dataset_csv,
                                                 representation_fields={"proteins": "SEQ",
                                                                       "ligands": "SUBSTRATES"},
                                                 instances_ids_field={"interaction": "ids"},
                                                 labels_field="LogSpActivity")

The MultiInputDataset converts everything into dictionaries to ensure that we are always processing unique sequences and not the same multiple times

In [4]:
multi_input_dataset.instances["proteins"]

{2: 'MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVLTSVKKAEQYLLENETTKNYLGIDGIPEFGRCTQELLFGKGSALINDKRARTAQTPGGTGALRVAADFLAKNTSVKRVWVSNPSWPNHKSVFNSAGLEVREYAYYDAENHTLDFDALINSLNEAQAGDVVLFHGCCHNPTGIDPTLEQWQTLAQLSVEKGWLPLFDFAYQGFARGLEEDAEGLRAFAAMHKELIVASSYSKNFGLYNERVGACTLVAADSETVDRAFSQMKAAIRANYSNPPAHGASVVATILSNDALRAIWEQELTDMRQRIQRMRQLFVNTLQEKGANRDFSFIIKQNGMFSFSGLTKEQVLRLREEFGVYAVASGRVNVAGMTPDNMAPLCEAIVAVL',
 1: 'MDYVTLASHAVRQYAPDQIFTASQRAKADAAALGEDAVINATLGECLDDDGKLMVLPTVERMIRQMPVEDICSYAPIAGIKGFNEAVQISLFGKCLDRFYVESVATPGGCGALRHAIWNFLNFGDALLTTNWIWGPYKNICEEHGRRMVTFDMFNRENTFNLEGMDRAIGEILAVQEQLLMILNTPANNPTGYSMTKQEMEQTVAILKKHAAANPDKNLTFCLDVSYIDFAGSFEESREIFDAIFDMPANTMTLLIFSMSKSYTMCGMRCGALVCLGSTAESAAVFKQAMSYSSRSTWSNAIHMAQKILVDINLNPEIRERVSQERAVFRNTITNRGRTFCAAAKEASLEICPYQYGYFVAIPCKNPVETARILMDQHIYVVPQAQGLRFSPCTVTTEKCRKAPAFIKAAMEQTQ',
 3: 'MFQKVDAYAGDPILTLMERFKEDPRSDKVNLSIGLYYNEDGIIPQLQAVAEAEARLNAQPHGASLYLPMEGLNCYRHAIAPLLFGADHPVLKQQRVATIQTLGGSGALKVGADFLKRYFPESGVWVSDPTWENHVAIFAGAGFEVSTYPWYDEATNGVRFNDLLATLK

In [5]:
multi_input_dataset.instances["ligands"]

{1: 'C(C(=O)O)N',
 6: 'C[C@@H](C(=O)O)N',
 5: 'CC(C)[C@@H](C(=O)O)N',
 3: 'CC(C)C[C@@H](C(=O)O)N',
 2: 'C1=C(NC=N1)C[C@@H](C(=O)O)N',
 4: 'CC(C)[C@@H](C(=O)O)*'}

## Standardize sequences

In [6]:
multi_input_dataset = ProteinStandardizer(n_jobs=50).fit_transform(multi_input_dataset, instance_type="proteins")

Featurizing: 100%|██████████| 3/3 [00:00<00:00, 261.25it/s]


In [7]:
multi_input_dataset.instances["proteins"]

{2: 'MFENITAAPADPILGLADLFRADERPGKINLGIGVYKDETGKTPVLTSVKKAEQYLLENETTKNYLGIDGIPEFGRCTQELLFGKGSALINDKRARTAQTPGGTGALRVAADFLAKNTSVKRVWVSNPSWPNHKSVFNSAGLEVREYAYYDAENHTLDFDALINSLNEAQAGDVVLFHGCCHNPTGIDPTLEQWQTLAQLSVEKGWLPLFDFAYQGFARGLEEDAEGLRAFAAMHKELIVASSYSKNFGLYNERVGACTLVAADSETVDRAFSQMKAAIRANYSNPPAHGASVVATILSNDALRAIWEQELTDMRQRIQRMRQLFVNTLQEKGANRDFSFIIKQNGMFSFSGLTKEQVLRLREEFGVYAVASGRVNVAGMTPDNMAPLCEAIVAVL',
 1: 'MDYVTLASHAVRQYAPDQIFTASQRAKADAAALGEDAVINATLGECLDDDGKLMVLPTVERMIRQMPVEDICSYAPIAGIKGFNEAVQISLFGKCLDRFYVESVATPGGCGALRHAIWNFLNFGDALLTTNWIWGPYKNICEEHGRRMVTFDMFNRENTFNLEGMDRAIGEILAVQEQLLMILNTPANNPTGYSMTKQEMEQTVAILKKHAAANPDKNLTFCLDVSYIDFAGSFEESREIFDAIFDMPANTMTLLIFSMSKSYTMCGMRCGALVCLGSTAESAAVFKQAMSYSSRSTWSNAIHMAQKILVDINLNPEIRERVSQERAVFRNTITNRGRTFCAAAKEASLEICPYQYGYFVAIPCKNPVETARILMDQHIYVVPQAQGLRFSPCTVTTEKCRKAPAFIKAAMEQTQ',
 3: 'MFQKVDAYAGDPILTLMERFKEDPRSDKVNLSIGLYYNEDGIIPQLQAVAEAEARLNAQPHGASLYLPMEGLNCYRHAIAPLLFGADHPVLKQQRVATIQTLGGSGALKVGADFLKRYFPESGVWVSDPTWENHVAIFAGAGFEVSTYPWYDEATNGVRFNDLLATLK

In [8]:
multi_input_dataset = DeepMolStandardizer(n_jobs=50).fit_transform(multi_input_dataset, "ligands")

Featurizing: 100%|██████████| 6/6 [00:00<00:00, 455.75it/s]


In [9]:
multi_input_dataset.instances["ligands"]

{1: '[H]OC(=O)C([H])([H])N([H])[H]',
 6: '[H]OC(=O)C([H])(N([H])[H])C([H])([H])[H]',
 5: '[H]OC(=O)C([H])(N([H])[H])C([H])(C([H])([H])[H])C([H])([H])[H]',
 3: '[H]OC(=O)C([H])(N([H])[H])C([H])([H])C([H])(C([H])([H])[H])C([H])([H])[H]',
 2: '[H]OC(=O)C([H])(N([H])[H])C([H])([H])c1c([H])nc([H])n1[H]',
 4: '*C([H])(C(=O)O[H])C([H])(C([H])([H])[H])C([H])([H])[H]'}

## Generating features and encodings

In [4]:
proteins_one_hot = Word2Vec().fit(multi_input_dataset, "proteins")
multi_input_dataset = proteins_one_hot.transform(multi_input_dataset, "proteins")

Word2Vec: 100%|██████████| 3/3 [00:00<00:00, 484.35it/s]


In [5]:
deepmol_wrapper = DeepMolDescriptors(n_jobs=8).fit(multi_input_dataset, instance_type="ligands")
multi_input_dataset = deepmol_wrapper.transform(multi_input_dataset, instance_type="ligands")

DeepMolDescriptors: 100%|██████████| 6/6 [00:00<00:00, 21.74it/s]


In [12]:
multi_input_dataset.X["proteins"]

array([[-0.0329832 , -0.01935539,  0.00307129, ..., -0.06316928,
         0.03729028, -0.05327919],
       [-0.0329832 , -0.01935539,  0.00307129, ..., -0.06316928,
         0.03729028, -0.05327919],
       [-0.0329832 , -0.01935539,  0.00307129, ..., -0.06316928,
         0.03729028, -0.05327919],
       ...,
       [-0.032417  , -0.01890334,  0.00405538, ..., -0.06444652,
         0.03711376, -0.05138807],
       [-0.032417  , -0.01890334,  0.00405538, ..., -0.06444652,
         0.03711376, -0.05138807],
       [-0.032417  , -0.01890334,  0.00405538, ..., -0.06444652,
         0.03711376, -0.05138807]], dtype=float32)

In [13]:
multi_input_dataset.X["ligands"]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [15]:
multi_input_dataset.y

array([[0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1]])

## Tokenizers

In [16]:
AtomLevelTokenizer().tokenize("CC[C@H](C)[C@@H](C(=O)O)N")

['C',
 'C',
 '[C@H]',
 '(',
 'C',
 ')',
 '[C@@H]',
 '(',
 'C',
 '(',
 '=',
 'O',
 ')',
 'O',
 ')',
 'N']

## Running models

In [6]:
input_size_proteins = multi_input_dataset.X["proteins"].shape[1]
input_size_compounds = multi_input_dataset.X["ligands"].shape[1]
model = BaselineModel(input_size_proteins, input_size_compounds, hidden_layers_proteins=[500, 250],
                      hidden_layers_compounds=[500, 500], hidden_layers_interaction=[500, 500])

wrapper = PyTorchModel(model=model, loss_function=nn.BCELoss(),
                               validation_metric=f1_score,
                               problem_type=BINARY, batch_size=50, epochs=100,
                               optimizer=Adam(model.parameters(), lr=0.0001), progress=50,
                               logger_path="small_dataset.log")
wrapper.fit(multi_input_dataset)

2023-03-05 19:37:14.356642: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 19:37:15.429206: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-05 19:37:15.429335: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-05 19:37:15.429350: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

BaselineModel(
  (dense_proteins1): Linear(in_features=512, out_features=1024, bias=True)
  (fc_proteins0): Linear(in_features=1024, out_features=500, bias=True)
  (fc_proteins1): Linear(in_features=500, out_features=250, bias=True)
  (dense_compounds1): Linear(in_features=2048, out_features=4096, bias=True)
  (fc_compounds0): Linear(in_features=4096, out_features=500, bias=True)
  (fc_compounds1): Linear(in_features=500, out_features=500, bias=True)
  (dense_interaction_layer1): Linear(in_features=750, out_features=4096, bias=True)
  (fc_interaction0): Linear(in_features=4096, out_features=500, bias=True)
  (fc_interaction1): Linear(in_features=500, out_features=500, bias=True)
  (final_layer): Linear(in_features=500, out_features=1, bias=True)
)

In [2]:
from torch import nn

params = {
    'features': 512,
    'lr': 0.001,
    'momentum': 0,
}

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, params['features']),
            nn.ReLU(),
            nn.Linear(params['features'], params['features']),
            nn.ReLU(),
            nn.Linear(params['features'], 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
wrapper = PyTorchModel(model=model, loss_function=nn.BCELoss(),
                               validation_metric=f1_score,
                               problem_type=BINARY, batch_size=50, epochs=100,
                               optimizer=Adam(model.parameters(), lr=0.0001), progress=50,
                               logger_path="small_dataset.log")
wrapper.fit(multi_input_dataset)